# Testing the addition to 3 new agents and functions

### Web Search (bing), Python and Weather Data

**ADMIN**

In [1]:
import autogen
from typing import Literal
import os

from pydantic import BaseModel, Field
from typing_extensions import Annotated
import requests
import autogen
from autogen.cache import Cache

config_list = [{
    'model': 'gpt-4o',
    'api_key': 'sk-proj-iQtcgUJOOf4n53Bs6uyqT3BlbkFJnEIqUeEwXjbjVMcDVqiz'
}]

llm_config={
    # "request_timeout": 600,
    # "seed": 42,
    "config_list": config_list,
    "temperature": 0
}
os.environ['BING_API_KEY'] = '7e534482ac5f46d3a2a979072b19e591'
bing_api_key = os.environ["BING_API_KEY"]


**DEFINE AGENTS**

In [2]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "group",
        "use_docker": False,},
    max_consecutive_auto_reply=10,
    system_message="""Reply TERMINATE if the task been solved at full satisfaction. 
    Otherwise, reply CONTINUE or the reason why the task is not solved yet. """,

)

In [3]:
current_weather_data_retriever = autogen.AssistantAgent(
    name="chatbot",
    # system_message="For weather forecasting tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    system_message = """You are the agent specializing in answering all weather related queries.
    When part of a group, please direct all such queries to yourself
    Please only use the functions you have been provided with. Reply TERMINATE when the task is done.""",
    llm_config=llm_config
    
)

In [4]:
engineer = autogen.AssistantAgent(
    name="engineer",
    system_message="""

    For coding tasks, only use the functions you have been provided with. 
    Reply TERMINATE when the task is done.
    
    """,
    llm_config=llm_config,
)

___
**DEFINE TOOLS & ASSIGN TO AGENTS**

In [5]:
import os
import requests

@user_proxy.register_for_execution()
@current_weather_data_retriever.register_for_llm(description="function answering all queries related to the weather")
def get_temperature_data(zipcode: str) -> dict:
    api_key = "864a9bb0b562e1c87e01b38880d5bee7"
    geocoding_url = "http://api.openweathermap.org/geo/1.0/zip"
    one_call_url = "https://api.openweathermap.org/data/3.0/onecall"
    
    # Step 1: Convert zipcode to geographic coordinates
    geocoding_params = {"zip": zipcode, "appid": api_key}
    geo_response = requests.get(geocoding_url, params=geocoding_params)
    if geo_response.status_code != 200:
        return {"error": "Failed to fetch geographic coordinates."}
    
    geo_data = geo_response.json()
    lat, lon = geo_data.get("lat"), geo_data.get("lon")
    
    # Step 2: Fetch weather data using One Call API
    one_call_params = {
        "lat": lat,
        "lon": lon,
        "exclude": "minutely,hourly,alerts",  # Exclude unnecessary data
        "appid": api_key,
        "units": "metric"  # Metric units; change to "imperial" if preferred
    }
    weather_response = requests.get(one_call_url, params=one_call_params)
    if weather_response.status_code != 200:
        return {"error": "Failed to fetch weather data."}
    
    weather_data = weather_response.json()
    current = weather_data.get("current", {})
    daily = weather_data.get("daily", [])

    # Extracting key data
    current_temp = current.get("temp")
    humidity = current.get("humidity")
    wind_speed = current.get("wind_speed")
    weather_description = current.get("weather", [{}])[0].get("description")
    
    daily_temps = [day.get("temp", {}).get("day") for day in daily]
    average_temp = sum(daily_temps) / len(daily_temps) if daily_temps else None

    daily_humidity = [day.get("humidity") for day in daily]
    average_humidity = sum(daily_humidity) / len(daily_humidity) if daily_humidity else None

    daily_wind_speed = [day.get("wind_speed") for day in daily]
    average_wind_speed = sum(daily_wind_speed) / len(daily_wind_speed) if daily_wind_speed else None

    return {
        "current_temperature": current_temp,
        "average_temperature": average_temp,
        "current_humidity": humidity,
        "average_humidity": average_humidity,
        "current_wind_speed": wind_speed,
        "average_wind_speed": average_wind_speed,
        "weather_description": weather_description
    }

In [6]:
# one way of registering functions is to use the register_for_llm and register_for_execution decorators
@user_proxy.register_for_execution()
@engineer.register_for_llm(description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log

In [7]:
from autogen.agentchat.contrib.web_surfer import WebSurferAgent  # noqa: E402

web_surfer = WebSurferAgent(
    "web_surfer",
    llm_config=llm_config,
    summarizer_llm_config=llm_config,
    browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key},
)

---

In [8]:
current_weather_data_retriever.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'function answering all queries related to the weather',
   'name': 'get_temperature_data',
   'parameters': {'type': 'object',
    'properties': {'zipcode': {'type': 'string', 'description': 'zipcode'}},
    'required': ['zipcode']}}}]

In [9]:
engineer.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'run cell in ipython and return the execution result.',
   'name': 'exec_python',
   'parameters': {'type': 'object',
    'properties': {'cell': {'type': 'string',
      'description': 'Valid Python cell to execute.'}},
    'required': ['cell']}}}]

**TEST AGENTS & FUNCTIONS**

In [10]:
# Example usage
zipcode = "95321"  # San Francisco, Zip Code 94131
weather_data = get_temperature_data(zipcode)
print(weather_data)

{'current_temperature': 23.16, 'average_temperature': 35.06875, 'current_humidity': 32, 'average_humidity': 15.125, 'current_wind_speed': 1.92, 'average_wind_speed': 4.6925, 'weather_description': 'clear sky'}


---
## Run Python & Web Search

---

In [11]:
task = "Can you get the current weather information for San Francisco, Zip Code 94550?, \
        and plot the returned values in a table using the python tool available to you\
        please answer this question only using the approrpriate tool when required\
        for example, please use the weather related tool for all weather related queries\
        please use the search tool only if specifically asked to search the web for answer"

In [12]:
groupchat = autogen.GroupChat(agents=[user_proxy, current_weather_data_retriever, engineer, web_surfer], \
                              messages=[], max_round=5)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [13]:
with Cache.disk() as cache:
    # start the conversation
    res = user_proxy.initiate_chat(
        manager, message=task, summary_method="reflection_with_llm", cache=cache, llm_config=llm_config
    )

user_proxy (to chat_manager):

Can you get the current weather information for San Francisco, Zip Code 94550?,         and plot the returned values in a table using the python tool available to you        please answer this question only using the approrpriate tool when required        for example, please use the weather related tool for all weather related queries        please use the search tool only if specifically asked to search the web for answer

--------------------------------------------------------------------------------

Next speaker: chatbot

chatbot (to chat_manager):

***** Suggested tool call (call_p8uU2hK13Pa0iU8CuwHphfqT): get_temperature_data *****
Arguments: 
{"zipcode": "94550"}
*************************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION get_temperature_data...
user_proxy (to chat_manager):

user_proxy

,Metric,Value
0,Current Temperature (°C),14.1
1,Average Temperature (°C),29.0825
2,Current Humidity (%),80
3,Average Humidity (%),30.0
4,Current Wind Speed (m/s),2.68
5,Average Wind Speed (m/s),6.98375
6,Weather Description,clear sky


user_proxy (to chat_manager):

user_proxy (to chat_manager):

***** Response from calling tool (call_f6TnUYTmccQ1lSklS8BfrSbh) *****
                     Metric      Value
0  Current Temperature (°C)       14.1
1  Average Temperature (°C)    29.0825
2      Current Humidity (%)         80
3      Average Humidity (%)       30.0
4  Current Wind Speed (m/s)       2.68
5  Average Wind Speed (m/s)    6.98375
6       Weather Description  clear sky
**********************************************************************

--------------------------------------------------------------------------------
